In [1]:
import pickle
import autograd.numpy as np
from viabel.family import mean_field_gaussian_variational_family
import pandas as pd

import matplotlib.pyplot as plt
import time

In [2]:
data = pd.read_csv("bodyfat.csv").drop("Density", axis=1)
data = data - data.mean()
y = data["Bodyfat"].values
# centre the response to adjust for no intercept
y = y - np.mean(y)

x = data.drop("Bodyfat", axis=1)
x['Intercept'] = 1

N = x.shape[0]
D = x.shape[1]

In [3]:
mc_samples = 100000
M = 100

klvi_probs_mc = np.zeros(shape=(D+1, M))
klvi_probs_sis = np.zeros(shape=(D+1, M))
klvi_probs_psis = np.zeros(shape=(D+1, M))

In [4]:
mf_gaussian_var_family = mean_field_gaussian_variational_family(D+1)

test

In [5]:
with open("./data/klvi_psis_"+str(1)+".pkl", 'rb') as f:
    [btemp, stemp, klvi_samples_temp, klvi_slw_temp, klvi_lw_temp, klvi_khat_temp, klvi_var_param] = pickle.load(f)

In [6]:
klvi_khat_temp

0.9345475644451805

get probs

In [7]:
start = time.time()
for i in range(M):
    start_loop = time.time()
    with open("./data/klvi_psis_"+str(i)+".pkl", 'rb') as f:
        [btemp, stemp, klvi_samples_temp, klvi_slw_temp, klvi_lw_temp, klvi_khat_temp, klvi_var_param] = pickle.load(f)
    
    scaled_weights_sis = np.exp(klvi_lw_temp - max(klvi_lw_temp))
    scaled_weights_psis = np.exp(klvi_slw_temp - max(klvi_slw_temp))
    khat_temp = klvi_khat_temp
    
    for j in range(D+1):
        prob_mc_temp = 0
        prob_sis_temp = 0
        prob_psis_temp = 0
        
        samples_temp = klvi_samples_temp[:][j]
        
        for k in range(mc_samples):
            if (j < D and samples_temp[k] < btemp[0][j]) or (j == D and np.log(samples_temp[k]) < np.log(stemp[0])):
                # unadjusted monte carlo
                prob_mc_temp += 1
                
                # sis
                prob_sis_temp += scaled_weights_sis[k]
                
                # psis
                prob_psis_temp += scaled_weights_psis[k]
                    
        klvi_probs_mc[j, i] = prob_mc_temp / mc_samples
        klvi_probs_sis[j, i] = prob_sis_temp / sum(scaled_weights_sis)
        klvi_probs_psis[j, i] = prob_psis_temp / sum(scaled_weights_psis)
    
    end_loop = time.time()
    print("i = %i, khat = %f, loop time = %f, time elapsed = %f" % (i, khat_temp, end_loop-start_loop, end_loop-start))

end = time.time()
print("Total time taken: %f" % (end-start))

i = 0, khat = 0.992865, loop time = 2.667227, time elapsed = 2.667227
i = 1, khat = 0.934548, loop time = 2.595607, time elapsed = 5.263823
i = 2, khat = 0.908968, loop time = 2.375682, time elapsed = 7.639504
i = 3, khat = 0.953332, loop time = 2.500172, time elapsed = 10.140678
i = 4, khat = 1.067131, loop time = 2.425998, time elapsed = 12.567677
i = 5, khat = 1.016520, loop time = 2.561328, time elapsed = 15.130005
i = 6, khat = 0.936410, loop time = 2.240788, time elapsed = 17.370793
i = 7, khat = 0.954992, loop time = 2.287742, time elapsed = 19.659535
i = 8, khat = 0.982760, loop time = 2.505281, time elapsed = 22.165817
i = 9, khat = 1.052040, loop time = 2.570091, time elapsed = 24.736909
i = 10, khat = 0.910678, loop time = 2.422688, time elapsed = 27.159596
i = 11, khat = 0.913164, loop time = 2.496269, time elapsed = 29.655866
i = 12, khat = 0.964149, loop time = 2.592809, time elapsed = 32.248674
i = 13, khat = 1.021445, loop time = 2.671223, time elapsed = 34.919898
i = 1

In [8]:
np.savetxt("./data/klvi_probs_mc.csv", klvi_probs_mc, delimiter=",")
np.savetxt("./data/klvi_probs_sis.csv", klvi_probs_sis, delimiter=",")
np.savetxt("./data/klvi_probs_psis.csv", klvi_probs_psis, delimiter=",")

In [9]:
chivi_probs_mc = np.zeros(shape=(D+1, M))
chivi_probs_sis = np.zeros(shape=(D+1, M))
chivi_probs_psis = np.zeros(shape=(D+1, M))

In [10]:
start = time.time()
for i in range(M):
    start_loop = time.time()
    with open("./data/chivi_psis_"+str(i)+".pkl", 'rb') as f:
        [btemp, stemp, chivi_samples_temp, chivi_slw_temp, chivi_lw_temp, chivi_khat_temp, chivi_var_param] = pickle.load(f)
    
    scaled_weights_sis = np.exp(chivi_lw_temp - max(chivi_lw_temp))
    scaled_weights_psis = np.exp(chivi_slw_temp - max(chivi_slw_temp))
    khat_temp = chivi_khat_temp
    
    for j in range(D+1):
        prob_mc_temp = 0
        prob_sis_temp = 0
        prob_psis_temp = 0
        
        samples_temp = chivi_samples_temp[:][j]
        
        for k in range(mc_samples):
            if (j < D and samples_temp[k] < btemp[0][j]) or (j == D and np.log(samples_temp[k]) < np.log(stemp[0])):
                # unadjusted monte carlo
                prob_mc_temp += 1
                
                # sis
                prob_sis_temp += scaled_weights_sis[k]
                
                # psis
                prob_psis_temp += scaled_weights_psis[k]
                    
        chivi_probs_mc[j, i] = prob_mc_temp / mc_samples
        chivi_probs_sis[j, i] = prob_sis_temp / sum(scaled_weights_sis)
        chivi_probs_psis[j, i] = prob_psis_temp / sum(scaled_weights_psis)
    
    end_loop = time.time()
    print("i = %i, khat = %f, loop time = %f, time elapsed = %f" % (i, khat_temp, end_loop-start_loop, end_loop-start))

end = time.time()
print("Total time taken: %f" % (end-start))

i = 0, khat = 1.948259, loop time = 2.684464, time elapsed = 2.684464
i = 1, khat = 1.946078, loop time = 2.492562, time elapsed = 5.178026
i = 2, khat = 2.105806, loop time = 2.500514, time elapsed = 7.678540
i = 3, khat = 1.936103, loop time = 2.161443, time elapsed = 9.839983
i = 4, khat = 2.041921, loop time = 2.571298, time elapsed = 12.412282
i = 5, khat = 2.100091, loop time = 2.861894, time elapsed = 15.275176
i = 6, khat = 1.905181, loop time = 2.638902, time elapsed = 17.915079
i = 7, khat = 1.925670, loop time = 2.825802, time elapsed = 20.741882
i = 8, khat = 1.951021, loop time = 2.605254, time elapsed = 23.347136
i = 9, khat = 1.930081, loop time = 2.516526, time elapsed = 25.863662
i = 10, khat = 2.032152, loop time = 2.502144, time elapsed = 28.365806
i = 11, khat = 2.124322, loop time = 2.699454, time elapsed = 31.065260
i = 12, khat = 1.976800, loop time = 2.381913, time elapsed = 33.448175
i = 13, khat = 1.951457, loop time = 2.741602, time elapsed = 36.190777
i = 14

In [11]:
np.savetxt("./data/chivi_probs_mc.csv", chivi_probs_mc, delimiter=",")
np.savetxt("./data/chivi_probs_sis.csv", chivi_probs_sis, delimiter=",")
np.savetxt("./data/chivi_probs_psis.csv", chivi_probs_psis, delimiter=",")